### This notebook prepares the downloaded historical stock data and calculates futures necessary to run predictions

Import libraries

In [21]:
import sys
import pandas as pd
sys.path.append('/helpers')
from dates import month_end
# data preparation dastaframe functions
from dataframes import rolling_max, cumulative_max, increase_decline_streak, previous_swing

Import the widgets

In [40]:
from notebook_widgets.data_prep import selectors
# display widgets
for s in selectors:
    display(selectors[s]['display'])

Dropdown(description='File:', options=('download_20221209_181319', 'download_20221209_181333', 'download_20221…

Dropdown(description='Stock code:', index=3, options=('ASX.AX', 'IJR', 'IVV', 'IVV.AX', 'VAF.AX', 'VAP.AX', 'V…

Import the selected download

In [41]:
selected_download = (
    pd.read_csv('/data/' + selectors['file']['display'].value + '.csv', parse_dates=['Date'])
    .query('stock_code == "{}"'.format(selectors['stock_code']['display'].value))
    .reset_index(drop=True)
)

# display number of records
print('Number of records: {}'.format(selected_download.shape[0]))

Number of records: 508


Let's reshape the dataset to contain a record for each day, e.g. weekends and public holidays included

In [42]:
_data = []

for subset in ['training', 'validation']: 

    # get the minimum and maximum years from the downloaded dataframe
    subset_df = (
        selected_download
        .query(f'subset == "{subset}"')
        .reset_index(drop=True)
    )
    
    min_date = pd.to_datetime(subset_df['Date']).min().replace(day=1)
    max_date = month_end(pd.to_datetime(subset_df['Date']).max())
    
    new_dates = pd.DataFrame(
        {
            'Date' : pd.date_range(start=min_date, end=max_date, freq='D').to_series().reset_index(drop=True)
        })
    
    subset_df = (
        selected_download
        .merge(new_dates, how='right', left_on='Date', right_on='Date')
        .assign(subset=subset)
        .ffill(axis=0)
        .bfill(axis=0)
    )
    _data.append(subset_df.copy())

# replace the downloaded dataframe with the upsampled one
selected_download = pd.concat(_data, ignore_index=True)

# display number of records
print('Number of records: {}'.format(selected_download.shape[0]))

Number of records: 731


Model target and features

| Target | Description |
| :--- | :--- |
| `adj_close_next_day` | adjusted closing price increase on the next day |


| Feature | Description |
| :--- | :--- |
| `highest_5`| Is the price highest in 5 days? |
| `highest_10`| Is the price highest in 10 days? |
| `highest_30`| Is the price highest in 30 days? |
| `highest_ever`| Is the price highest ever seen? |
| `is_monday`| Is it a Monday? |
| `is_friday`| Is it a Friday? |
| `increase_streak`| Number of consecutive days the close price has been increasing, i.e. higher than the day before|
| `decline_streak`| Number of consecutive days the close price has been decreasing, i.e. lower than the day before|
| `previous_swing`| Difference between the high and low price for the previous day|

We start by calculating the target `adj_close_next_day` first

In [43]:
selected_download = (
    selected_download
    .assign(adj_close_next_day = 1 / (selected_download['Adj Close'].pct_change(periods=-1) + 1))
)

Calculate `1 or 0` flag if the price is the highest in 5, 10 and 30 days, e.g. `highest_5`, `highest_10` and `highest_30`

Calculate `1 or 0` flag if the price is the highest ever or cumulative max, e.g. `highest_ever`

In [44]:
selected_download = (
    selected_download
    .pipe(rolling_max, 'Adj Close', 5)
    .pipe(rolling_max, 'Adj Close', 10)
    .pipe(rolling_max, 'Adj Close', 30)
    .pipe(cumulative_max, 'Adj Close')
)

Calculate date features `is_monday` and `is_friday`

In [45]:
selected_download = (
    selected_download
    .assign(is_monday=selected_download['Date'].dt.weekday.apply(lambda ser : 1 if ser == 0 else 0))
    .assign(is_friday=selected_download['Date'].dt.weekday.apply(lambda ser : 1 if ser == 4 else 0))    
)

Calculate date increase and decline streaks, e.g. number of days the price has been increasing `increase_streak` or declining `decline_streak` 

In [46]:
selected_download = (
    selected_download
    .pipe(increase_decline_streak, 'Adj Close')
)

Calculate difference between the High and Low price for the previous day `previous_swing`

In [47]:
selected_download = (
    selected_download
    .pipe(previous_swing, ('High', 'Low'))
)

Save the prepared data into `/data`

In [50]:
# generate a file name
_ = lambda x, y : 'features_' + x.split('download_')[1] + f'_{y}.csv'
features_file_name = _(selectors['file']['display'].value, selectors['stock_code']['display'].value)

# output the file
(selected_download
 .drop(columns=['stock_code', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'])
 .loc[~selected_download['adj_close_next_day'].isna()]
 .reset_index(drop=True)
 .to_csv(f'../data/{features_file_name}', index=False)
)

print(f"Features file save as '{features_file_name}'")

Features file save as 'features_20221209_181319_IVV.AX.csv'
